In [1]:
import pandas as pd

MA1 = pd.DataFrame()
for chunk in pd.read_csv("../ssd_data/combined_by_diskmodel/combined_MA1.csv", chunksize=1000000):
    MA1 = pd.concat([MA1, chunk], ignore_index=True)
MA1

,index,disk_id,ds,model,n_1,r_1,n_5,r_5,n_9,r_9,...,n_233,n_241,r_241,n_242,r_242,r_244,n_175,r_175,n_232,label
0,45,10045,20190101,MA1,130.0,4.294967e+09,97.0,0.0,100.0,38288.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,9.752124e+11,NaN,0
1,50,10048,20190101,MA1,130.0,4.294967e+09,96.0,0.0,100.0,38288.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,9.752134e+11,NaN,0
2,69,10066,20190101,MA1,130.0,4.294967e+09,96.0,0.0,100.0,46396.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,1.185723e+12,NaN,0
3,86,10083,20190101,MA1,130.0,4.294967e+09,98.0,0.0,100.0,39978.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,1.014085e+12,NaN,0
4,129,10122,20190101,MA1,130.0,4.294967e+09,97.0,0.0,100.0,45592.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,1.151518e+12,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4744310,11267559,9838,20190630,MA1,130.0,4.294967e+09,97.0,0.0,100.0,43197.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,1.086756e+12,NaN,0
4744311,11267567,9842,20190630,MA1,130.0,5.896511e+06,98.0,0.0,100.0,42705.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,1.078410e+12,NaN,0
4744312,11267600,9878,20190630,MA1,130.0,4.294967e+09,96.0,0.0,100.0,41465.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.048160e+12,NaN,0
4744313,11267700,9970,20190630,MA1,130.0,4.294967e+09,96.0,0.0,100.0,46623.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,1.164486e+12,NaN,0


In [2]:
del(MA1['index'])

In [3]:
# null값이 있는 열을 제거하는 코드
def remove_null_columns(df):
    # null인 컬럼을 확인하고, 모든 값이 null인 컬럼을 찾습니다.
    null_columns = df.columns[df.isnull().all()]
    
    # 모든 값이 null인 컬럼을 DataFrame에서 제거합니다.
    df = df.drop(null_columns, axis=1)
    
    return df

In [4]:
# 하나의 unique한 값으로만 이루어진 컬럼을 제거하는 코드
def remove_single_value_columns(df):
    single_value_cols = []
    for col in df.columns:
        unique_values = df[col].nunique()
        if unique_values == 1:
            single_value_cols.append(col)

    df_filtered = df.drop(single_value_cols, axis=1)
    return df_filtered


MA1을 기준으로 다시 컬럼이 NULL값이 있는 열과 단일 변수로 이루어진 열을 제거한다.

In [5]:
MA1 = remove_null_columns(MA1)
MA1 = remove_single_value_columns(MA1)

In [6]:
MA1=MA1.fillna(0)

디스크가 fail된 아이디를 가져와서 그 fail된 디스크아이디를 가진 모든 행을 따로 만들어서 전처리를 구현한다.

In [7]:
# 라벨이 1인 행을 필터링합니다.
selected_disk_model_1 = MA1[MA1['label'] == 1]

# 모델, disk_id, 그리고 날짜 순으로 정렬합니다.
selected_disk_model_1 = selected_disk_model_1.sort_values(by=['disk_id', 'ds'])

# disk_id와 model 열만을 선택하여 새로운 데이터 프레임에 저장합니다.
selected_disk_model_1 = selected_disk_model_1[['disk_id']].drop_duplicates().reset_index(drop=True)

# 결과 데이터 프레임을 출력합니다.
selected_disk_model_1

,disk_id
0,520
1,1156
2,1224
3,1225
4,1230
...,...
270,42763
271,42764
272,42767
273,42815


In [8]:
# disk_ids 데이터프레임에 있는 disk_id와 일치하는 행을 MA1 데이터프레임에서 불러옴
selected_label_1 = MA1[MA1['disk_id'].isin(selected_disk_model_1['disk_id'])]

# ds 컬럼에 따라 정렬
selected_label_1 = selected_label_1.sort_values(by=['disk_id','ds'])
# 결과 출력
selected_label_1

,disk_id,ds,r_1,n_5,r_5,r_9,r_12,n_171,r_171,n_172,...,r_194,r_195,n_196,r_196,r_197,r_198,r_199,n_175,r_175,label
821858,520,20190130,9.376596e+08,96.0,0.0,38191.0,27.0,100.0,0.0,100.0,...,16.0,33733630.0,100.0,0.0,0.0,0.0,0.0,100.0,9.793172e+11,0
858376,520,20190131,1.624577e+09,96.0,0.0,38215.0,27.0,100.0,0.0,100.0,...,17.0,33733726.0,100.0,0.0,0.0,0.0,0.0,100.0,9.794111e+11,0
873871,520,20190201,2.014090e+09,96.0,0.0,38239.0,27.0,100.0,0.0,100.0,...,17.0,33733759.0,100.0,0.0,0.0,0.0,0.0,100.0,9.795051e+11,0
907339,520,20190202,2.812968e+09,96.0,0.0,38263.0,27.0,100.0,0.0,100.0,...,16.0,33733774.0,100.0,0.0,0.0,0.0,0.0,100.0,9.795990e+11,0
943568,520,20190203,3.442008e+09,96.0,0.0,38287.0,27.0,100.0,0.0,100.0,...,15.0,33733798.0,100.0,0.0,0.0,0.0,0.0,100.0,9.796929e+11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4643066,42824,20190626,4.294967e+09,94.0,0.0,48860.0,46.0,100.0,0.0,100.0,...,17.0,884843672.0,100.0,0.0,0.0,0.0,0.0,100.0,1.241470e+12,0
4668803,42824,20190627,4.294967e+09,94.0,0.0,48884.0,46.0,100.0,0.0,100.0,...,17.0,884886553.0,100.0,0.0,0.0,0.0,0.0,100.0,1.241564e+12,0
4694939,42824,20190628,4.294967e+09,94.0,0.0,48908.0,46.0,100.0,0.0,100.0,...,18.0,884924703.0,100.0,0.0,0.0,0.0,0.0,100.0,1.241658e+12,0
4702639,42824,20190629,4.294967e+09,94.0,0.0,48932.0,46.0,100.0,0.0,100.0,...,18.0,884965991.0,100.0,0.0,0.0,0.0,0.0,100.0,1.241751e+12,0


그리고 fail된 디스크만 있는 dataframe에서 fail이후에도 디스크가 돌아가는 것이 있어서 그 행들을 다 제거하는 과정을 거쳤다.

In [9]:
remove_indexes = []
for index, row in selected_label_1.iterrows():
    if row['label'] == 1:
        disk_id = row['disk_id']
        ds = row['ds']
        remove_indexes.extend(selected_label_1.index[(selected_label_1['disk_id'] == disk_id) & (selected_label_1['ds'] > ds)].tolist())

# 중복된 인덱스를 제거하여 제거할 행들을 구함
remove_indexes = list(set(remove_indexes))

# 제거할 행을 제외한 최종 DataFrame을 생성
del_fail_after_row = selected_label_1.drop(index=remove_indexes)

# 결과 출력
print(del_fail_after_row)

        disk_id        ds           r_1   n_5  r_5      r_9  r_12  n_171  \
821858      520  20190130  9.376596e+08  96.0  0.0  38191.0  27.0  100.0   
858376      520  20190131  1.624577e+09  96.0  0.0  38215.0  27.0  100.0   
873871      520  20190201  2.014090e+09  96.0  0.0  38239.0  27.0  100.0   
907339      520  20190202  2.812968e+09  96.0  0.0  38263.0  27.0  100.0   
943568      520  20190203  3.442008e+09  96.0  0.0  38287.0  27.0  100.0   
...         ...       ...           ...   ...  ...      ...   ...    ...   
173863    42824  20190107  4.294967e+09  94.0  0.0  44799.0  46.0  100.0   
211513    42824  20190108  4.294967e+09  94.0  0.0  44823.0  46.0  100.0   
224212    42824  20190109  4.294967e+09  94.0  0.0  44847.0  46.0  100.0   
277435    42824  20190110  4.294967e+09  94.0  0.0  44871.0  46.0  100.0   
302808    42824  20190111  4.294967e+09  94.0  0.0  44895.0  46.0  100.0   

        r_171  n_172  ...  r_194        r_195  n_196  r_196  r_197  r_198  \
821858    

In [10]:
selected_label_1[(selected_label_1['disk_id']==42824) & (selected_label_1['label']==1)]

,disk_id,ds,r_1,n_5,r_5,r_9,r_12,n_171,r_171,n_172,...,r_194,r_195,n_196,r_196,r_197,r_198,r_199,n_175,r_175,label
302808,42824,20190111,4.294967e+09,94.0,0.0,44895.0,46.0,100.0,0.0,100.0,...,18.0,872922346.0,100.0,0.0,0.0,0.0,0.0,100.0,1.138653e+12,1


그 뒤에 30일 이전에 대해서 라벨링을 한다.
이때 주의해야할 점이 많은 디스크들이 fail되기 이전에 30일전이 연속적으로 있는게 아니라 중간에 비어있는 날짜들이 있다.
그래서 날짜를 기준으로 한달전을 label을 바꾼다. 만약 30일 전이 25일로 이루어져있으면 25개가 1로 라벨링 되는것이다.

In [11]:
import pandas as pd

# 'ds' 컬럼을 datetime 형태로 변환
del_fail_after_row['ds'] = pd.to_datetime(del_fail_after_row['ds'], format='%Y%m%d')

# 모든 행에 대해 반복
for index, row in del_fail_after_row.iterrows():
    # label 값이 1인 경우
    if row['label'] == 1:
        # 해당 행의 disk_id
        current_disk_id = row['disk_id']
        # 해당 행의 날짜
        current_date = row['ds']
        # 30일 이전의 날짜 계산
        days_before = current_date - pd.Timedelta(days=30)
        
        # 같은 disk_id이면서, 날짜가 30일 이내인 행들을 찾아서 label 값을 1로 설정
        del_fail_after_row.loc[(del_fail_after_row['disk_id'] == current_disk_id) &
                             (del_fail_after_row['ds'] <= current_date) &
                             (del_fail_after_row['ds'] > days_before), 'label'] = 1

In [12]:
del_fail_after_row[del_fail_after_row['label']==1]

,disk_id,ds,r_1,n_5,r_5,r_9,r_12,n_171,r_171,n_172,...,r_194,r_195,n_196,r_196,r_197,r_198,r_199,n_175,r_175,label
2813978,520,2019-04-10,4.294967e+09,96.0,0.0,39863.0,27.0,100.0,0.0,100.0,...,16.0,33735621.0,100.0,0.0,0.0,0.0,0.0,100.0,1.022235e+12,1
2825809,520,2019-04-11,4.294967e+09,96.0,0.0,39887.0,27.0,100.0,0.0,100.0,...,16.0,33735630.0,100.0,0.0,0.0,0.0,0.0,100.0,1.022329e+12,1
2862621,520,2019-04-12,4.294967e+09,96.0,0.0,39911.0,27.0,100.0,0.0,100.0,...,16.0,33735639.0,100.0,0.0,0.0,0.0,0.0,100.0,1.022423e+12,1
2869229,520,2019-04-13,4.294967e+09,96.0,0.0,39935.0,27.0,100.0,0.0,100.0,...,16.0,33735656.0,100.0,0.0,0.0,0.0,0.0,100.0,1.022517e+12,1
2904832,520,2019-04-14,4.294967e+09,96.0,0.0,39959.0,27.0,100.0,0.0,100.0,...,16.0,33735772.0,100.0,0.0,0.0,0.0,0.0,100.0,1.022611e+12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173863,42824,2019-01-07,4.294967e+09,94.0,0.0,44799.0,46.0,100.0,0.0,100.0,...,18.0,872307910.0,100.0,0.0,0.0,0.0,0.0,100.0,1.138277e+12,1
211513,42824,2019-01-08,4.294967e+09,94.0,0.0,44823.0,46.0,100.0,0.0,100.0,...,18.0,872462608.0,100.0,0.0,0.0,0.0,0.0,100.0,1.138371e+12,1
224212,42824,2019-01-09,4.294967e+09,94.0,0.0,44847.0,46.0,100.0,0.0,100.0,...,18.0,872613656.0,100.0,0.0,0.0,0.0,0.0,100.0,1.138465e+12,1
277435,42824,2019-01-10,4.294967e+09,94.0,0.0,44871.0,46.0,100.0,0.0,100.0,...,18.0,872767350.0,100.0,0.0,0.0,0.0,0.0,100.0,1.138559e+12,1


In [13]:
del_fail_after_row['label'].value_counts()

label
0    7248
1    5336
Name: count, dtype: int64

In [12]:
selected_label_1[(selected_label_1['disk_id']==29444) & (selected_label_1['label']==1)]

,disk_id,ds,r_1,n_5,r_5,r_9,r_12,n_171,r_171,n_172,...,r_194,r_195,n_196,r_196,r_197,r_198,r_199,n_175,r_175,label
1517953,29444,20190224,4.294967e+09,97.0,2.0,37820.0,32.0,100.0,2.0,100.0,...,15.0,697372681.0,99.0,2.0,0.0,0.0,0.0,100.0,9.581584e+11,1


In [13]:
selected_label_1[(selected_label_1['disk_id']==1156) & (selected_label_1['label']==1)]

,disk_id,ds,r_1,n_5,r_5,r_9,r_12,n_171,r_171,n_172,...,r_194,r_195,n_196,r_196,r_197,r_198,r_199,n_175,r_175,label
1621336,1156,20190228,4.294967e+09,98.0,0.0,40012.0,0.0,0.0,0.0,0.0,...,19.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,1.013887e+12,1


fail된 디스크가 275개라서 healthy한 디스크도 275개를 가져와서 병합하는 작업을 거친다.

In [16]:
import random

# 시드 값을 설정합니다.
random.seed(42)

# 라벨이 1인 행을 필터링합니다.
selected_disk_model_0 = MA1[MA1['label'] == 0]

# disk_id만 추출
selected_disk_model_0 = selected_disk_model_0['disk_id'].drop_duplicates()
selected_disk_model_0 = list(selected_disk_model_0)
selected_disk_model_0

# 275개를 랜덤하게 선택
selected_disk_model_0 = random.sample(selected_disk_model_0, 275)

In [17]:
# disk_ids 데이터프레임에 있는 disk_id와 일치하는 행을 MA1 데이터프레임에서 불러옴
selecetd_label_0= MA1[MA1['disk_id'].isin(selected_disk_model_0)]

# ds 컬럼에 따라 정렬
selecetd_label_0= selecetd_label_0.sort_values(by='ds')


# 'ds' 컬럼을 datetime 형태로 변환
selecetd_label_0['ds'] = pd.to_datetime(selecetd_label_0['ds'], format='%Y%m%d')

# 결과 출력
selecetd_label_0

,disk_id,ds,r_1,n_5,r_5,r_9,r_12,n_171,r_171,n_172,...,r_194,r_195,n_196,r_196,r_197,r_198,r_199,n_175,r_175,label
37,11021,2019-01-01,4.294967e+09,95.0,0.0,46679.0,33.0,100.0,0.0,100.0,...,18.0,9.717908e+08,100.0,0.0,0.0,0.0,0.0,100.0,1.185953e+12,0
17089,14869,2019-01-01,4.294967e+09,97.0,0.0,37640.0,0.0,0.0,0.0,0.0,...,14.0,0.000000e+00,100.0,0.0,0.0,0.0,0.0,100.0,9.581584e+11,0
17219,19415,2019-01-01,4.294967e+09,98.0,0.0,36844.0,41.0,100.0,0.0,100.0,...,12.0,3.701852e+07,100.0,0.0,0.0,0.0,0.0,100.0,9.410302e+11,0
17261,20740,2019-01-01,4.294967e+09,98.0,0.0,37172.0,41.0,100.0,0.0,100.0,...,14.0,1.262730e+07,100.0,0.0,0.0,0.0,0.0,100.0,9.453267e+11,0
17335,22839,2019-01-01,4.294967e+09,98.0,0.0,40441.0,74.0,100.0,0.0,100.0,...,14.0,7.784229e+08,100.0,0.0,0.0,0.0,0.0,100.0,1.009346e+12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4728727,34800,2019-06-30,4.294967e+09,97.0,0.0,42525.0,0.0,0.0,0.0,0.0,...,16.0,0.000000e+00,100.0,0.0,0.0,0.0,0.0,100.0,1.065720e+12,0
4728745,35176,2019-06-30,4.294967e+09,98.0,0.0,44852.0,51.0,100.0,0.0,100.0,...,14.0,1.627484e+09,100.0,0.0,0.0,0.0,0.0,100.0,1.134003e+12,0
4728842,38166,2019-06-30,4.294967e+09,98.0,2.0,42603.0,0.0,0.0,0.0,0.0,...,15.0,0.000000e+00,99.0,2.0,0.0,0.0,0.0,100.0,1.082352e+12,0
4731751,24294,2019-06-30,4.294967e+09,92.0,0.0,50139.0,25.0,100.0,0.0,100.0,...,16.0,2.237487e+08,100.0,0.0,0.0,0.0,0.0,100.0,1.275836e+12,0


In [21]:
from tqdm import tqdm

# final_df와 final_df2를 합칩니다.
final_df = pd.concat([del_fail_after_row, selecetd_label_0])

# model, disk_id, date를 기준으로 그룹화하고 shuffle합니다.
grouped = final_df.groupby(['disk_id'])
shuffled_groups = random.sample(list(grouped), len(grouped))

# 진행 상황을 표시하기 위해 tqdm 사용
shuffled_df = pd.DataFrame()
for _, group in tqdm(shuffled_groups):
    shuffled_df = pd.concat([shuffled_df, group])

# 최종 결과를 출력합니다.
shuffled_df

100%|██████████| 548/548 [00:01<00:00, 395.70it/s] 


,disk_id,ds,r_1,n_5,r_5,r_9,r_12,n_171,r_171,n_172,...,r_194,r_195,n_196,r_196,r_197,r_198,r_199,n_175,r_175,label
8241,2084,2019-01-01,4.294967e+09,96.0,0.0,40971.0,34.0,100.0,0.0,100.0,...,19.0,855971.0,100.0,0.0,0.0,0.0,0.0,100.0,1.039568e+12,0
34055,2084,2019-01-02,4.294967e+09,96.0,0.0,40999.0,34.0,100.0,0.0,100.0,...,17.0,855972.0,100.0,0.0,0.0,0.0,0.0,100.0,1.039678e+12,0
62125,2084,2019-01-03,4.294967e+09,96.0,0.0,41023.0,34.0,100.0,0.0,100.0,...,19.0,855972.0,100.0,0.0,0.0,0.0,0.0,100.0,1.039771e+12,0
97011,2084,2019-01-04,4.294967e+09,96.0,0.0,41047.0,34.0,100.0,0.0,100.0,...,18.0,855973.0,100.0,0.0,0.0,0.0,0.0,100.0,1.039865e+12,0
118409,2084,2019-01-05,4.294967e+09,96.0,0.0,41071.0,34.0,100.0,0.0,100.0,...,17.0,855975.0,100.0,0.0,0.0,0.0,0.0,100.0,1.039959e+12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573311,5491,2019-01-21,4.294967e+09,94.0,0.0,47222.0,49.0,100.0,0.0,100.0,...,14.0,690584521.0,100.0,0.0,0.0,0.0,0.0,100.0,1.177373e+12,0
594743,5491,2019-01-22,4.294967e+09,94.0,0.0,47246.0,49.0,100.0,0.0,100.0,...,14.0,691545911.0,100.0,0.0,0.0,0.0,0.0,100.0,1.177467e+12,0
628612,5491,2019-01-23,4.294967e+09,94.0,0.0,47270.0,49.0,100.0,0.0,100.0,...,14.0,692509923.0,100.0,0.0,0.0,0.0,0.0,100.0,1.181196e+12,0
2249249,5491,2019-03-21,4.294967e+09,94.0,0.0,48633.0,49.0,100.0,0.0,100.0,...,17.0,733131101.0,100.0,0.0,0.0,0.0,0.0,100.0,1.215629e+12,1


In [22]:
shuffled_df['label'].value_counts()

label
0    47426
1     5338
Name: count, dtype: int64

요약하자면 fail된 디스크에서 fail이후에 있는 디스크가 있는 행을 제거하고 fail됐을때의 날짜의 한달전을 <br>
기준으로 label을 1로 만들고 fail된 디스크만큼 healthy한 디스크를 가져와서 병합하는 작업을 거쳤다.<br>
그리고 그렇게 병합한 dataframe을 final_data_MA1_after_deleted.csv에 저장한다. 

In [23]:
shuffled_df.to_csv('../ssd_data/final_data/final_data_MA1_after_deleted.csv',index=False)